# Siamese-LSTM with Features(tm+nlp)
- Plus features in `feature_nlp.csv` and `feature_tm.csv` from previous work to leaky feature in Siamese_LSTM baseline.
- Change: adding around 40 more features to leaky features, and doubled N_DENSE in Dense layer. (from N_DENSE/2 to N_DENSE).

    
- **Output:**
  - `lstm_featured.csv`: 4 epoch, score: 0.16515
    - 1st epoch: loss: 0.2489 - acc: 0.8280 - val_loss: 0.1927 - val_acc: 0.8630
    - last epoch: loss: 0.1790 - acc: 0.8714 - val_loss: 0.1700 - val_acc: 0.8723

In [1]:
import os
import re
import csv
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from string import punctuation
from collections import defaultdict
# from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

# Keras package
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, Embedding, Dropout, Activation, LSTM, Lambda
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.pooling import GlobalAveragePooling1D
import keras.backend as K


Using TensorFlow backend.


In [2]:
# Hyperparameter defination

# Use the following instructions to download glove and unzip it, if already installed, just comment them.
# !wget http://nlp.stanford.edu/data/glove.840B.300d.zip
# !unzip glove.840B.300d.zip
EMBEDDING_FILE = 'glove.840B.300d.txt'

TRAIN_DATA_FILE = 'Data/train.csv'
TEST_DATA_FILE = 'Data/test.csv'

MAX_SEQUENCE_LENGTH = 60  
MAX_NUM_WORDS = 200000  # There are about 201000 unique words in training dataset, 200000 is enough for tokenization
EMBEDDING_DIM = 300  # word-embedded-vector dimension(300 is for 'glove.840B.300d')
VALIDATION_SPLIT_RATE = 0.1 
N_HIDDEN = 250
N_DENSE = 120
DROPOUT_RATE_LSTM = 0.33  # drop-out possibility, random set to avoid outfitting  # 0.20
DROUPOUT_RATE_DENSE = 0.33  

VERSION = 'Temp/lstm_featured'
print('LSTM Stucture:')
print('Num_Lstm:', N_HIDDEN)
print('Num_Dense:', N_DENSE)
print('Dropout rate in LSTM layer:', DROPOUT_RATE_LSTM) 
print('Dropout rate in Dense layer::', DROUPOUT_RATE_DENSE)

ACTIVE_FUNC = 'relu'
re_weight = True  # whether to re-weight classes to fit the 17.4% share in test set


LSTM Stucture:
Num_Lstm: 250
Num_Dense: 120
Dropout rate in LSTM layer: 0.33
Dropout rate in Dense layer:: 0.33


In [3]:
# Create word embedding dictionary from 'glove.840B.300d.txt', {key:value} is {word: glove vector(300,)}
print('Create word embedding dictionary')

embeddings_index = {}  # the output dictionary
f = open(EMBEDDING_FILE, encoding='utf-8')
for line in f:  # tqdm
    values = line.split()
    word = ''.join(values[:-300])   
    coefs = np.asarray(values[-300:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found {} word vectors of glove.'.format(len(embeddings_index)))


# Preprocess text in dataset
print('Processing text dataset')

def text_to_wordlist(text, remove_stopwords=False, stem_words=False):
    # Clean the text, with the option to remove stopwords and to stem words.
    
    # Convert words to lower case and split them
    text = text.lower().split()

    # Optionally, remove stop words
    if remove_stopwords:
        stop_words = set(stopwords.words("english"))
        text = [w for w in text if not w in stop_words]
    
    text = " ".join(text)

    # Use re to clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    # text = re.sub(r"\0s", "0", text) # It doesn't make sense to me
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    # or nltk package
    # if lemma:
    #    text = text.split()
    #    wn = nltk.WordNetLemmatizer()
    #    lemm_words = [wn.lemmatize(word) for word in text]
    #    text = " ".join(lemm_words)
    
    # Return a list of words
    return(text)

# Load training data and process with text_to_wordlist (Preprocessing)
train_texts_1 = []  # the preprocessed text of q1
train_texts_2 = []  # the preprocessed text of q2
train_labels = []  # training labels

df_train = pd.read_csv(TRAIN_DATA_FILE, encoding='utf-8')  # the original training data
df_train = df_train.fillna('empty')
train_q1 = df_train.question1.values  # the original text of q1
train_q2 = df_train.question2.values  # the original text fo q2
train_labels = df_train.is_duplicate.values  # the original label('is_duplicate')

for text in train_q1:
    train_texts_1.append(text_to_wordlist(text, remove_stopwords=False, stem_words=False))
    
for text in train_q2:
    train_texts_2.append(text_to_wordlist(text, remove_stopwords=False, stem_words=False))

print('{} texts are found in train.csv'.format(len(train_texts_1)))

# Load testing data and process with text_to_wordlist (Preprocessing)
test_texts_1 = []  # the preprocessed text of q1_test
test_texts_2 = []  # the preprocessed text of q2_test
test_ids = []  # id..

df_test = pd.read_csv(TEST_DATA_FILE, encoding='utf-8')  # the original testing data
df_test = df_test.fillna('empty')
test_q1 = df_test.question1.values  # the original text of q1_test
test_q2 = df_test.question2.values  # the original text of q2_test
test_ids = df_test.test_id.values  # id..

for text in test_q1:
    test_texts_1.append(text_to_wordlist(text, remove_stopwords=False, stem_words=False))
    
for text in test_q2:
    test_texts_2.append(text_to_wordlist(text, remove_stopwords=False, stem_words=False))
    
print('{} texts are found in test.csv'.format(len(test_texts_1)))


Create word embedding dictionary
Found 2195892 word vectors of glove.
Processing text dataset
404290 texts are found in train.csv
2345796 texts are found in test.csv


In [4]:
# Keras.Tokenize for all text:
# First construct a Tokenizer()
# Then use tokenizer_on_texts() method to learn the dictionary of the corpus(all texts(sentences)). We can use .word_index to map between the each word (distinct) with the corresponding number.
# Then use text_to_sequence() method to transfer every text(sentence) in texts into sequences of word_indexes.
# Then add the same length by padding method: padding_sequences().
# Finally use the embedding layer in keras to carry out a vectorization, and input it into LSTM.

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(train_texts_1 + train_texts_2 + test_texts_1 + test_texts_2)  # generate a token dictionary, 

train_sequences_1 = tokenizer.texts_to_sequences(train_texts_1)  # sequence of q1
train_sequences_2 = tokenizer.texts_to_sequences(train_texts_2)  # sequence of q2
test_sequences_1 = tokenizer.texts_to_sequences(test_texts_1)  # sequence of q1_test
test_sequences_2 = tokenizer.texts_to_sequences(test_texts_2)  # sequence of q2_test

word_index = tokenizer.word_index
print('{} unique tokens are found'.format(len(word_index)))

# Pad all train with Max_Sequence_Length: 60
train_data_1 = pad_sequences(train_sequences_1, maxlen=MAX_SEQUENCE_LENGTH)  # padded_sequence of q1 as train_data
train_data_2 = pad_sequences(train_sequences_2, maxlen=MAX_SEQUENCE_LENGTH)  # padded_sequence of q2 as train_data
print('Shape of train data tensor:', train_data_1.shape)
print('Shape of train labels tensor:', train_labels.shape)

# Pad all test with Max_Sequence_Length
test_data_1 = pad_sequences(test_sequences_1, maxlen=MAX_SEQUENCE_LENGTH)  # padded_sequence of q1_test as test_data
test_data_2 = pad_sequences(test_sequences_2, maxlen=MAX_SEQUENCE_LENGTH)  # padded_sequence of q2_test as test_data
print('Shape of test data vtensor:', test_data_2.shape)
print('Shape of test ids tensor:', test_ids.shape)


120499 unique tokens are found
Shape of train data tensor: (404290, 60)
Shape of train labels tensor: (404290,)
Shape of test data vtensor: (2345796, 60)
Shape of test ids tensor: (2345796,)


In [5]:
questions = pd.concat([df_train[['question1', 'question2']], df_test[['question1', 'question2']]], axis=0).reset_index(drop='index')
q_dict = defaultdict(set)
for i in range(questions.shape[0]):
        q_dict[questions.question1[i]].add(questions.question2[i])
        q_dict[questions.question2[i]].add(questions.question1[i])

def q1_freq_train(row):
    return len(q_dict.get(row[3]))
    # return(len(q_dict[row['question1']]))

def q2_freq_train(row):
    return len(q_dict.get(row[4]))
    # return(len(q_dict[row['question2']]))

def q1_q2_intersect_train(row):
    return(len(set(q_dict.get(row[3])).intersection(set(q_dict.get(row[4])))))
    # return(len(set(q_dict[row['question1']]).intersection(set(q_dict[row['question2']]))))

def q1_freq_test(row):
    return len(q_dict.get(row[1]))
    # return(len(q_dict[row['question1']]))

def q2_freq_test(row):
    return len(q_dict.get(row[2]))
    # return(len(q_dict[row['question2']]))

def q1_q2_intersect_test(row):
    return(len(set(q_dict.get(row[1])).intersection(set(q_dict.get(row[2])))))
    # return(len(set(q_dict[row['question1']]).intersection(set(q_dict[row['question2']]))))

df_train['q1_q2_intersect'] = df_train.apply(q1_q2_intersect_train, axis=1, raw=True)
df_train['q1_freq'] = df_train.apply(q1_freq_train, axis=1, raw=True)
df_train['q2_freq'] = df_train.apply(q2_freq_train, axis=1, raw=True)

df_test['q1_q2_intersect'] = df_test.apply(q1_q2_intersect_test, axis=1, raw=True)
df_test['q1_freq'] = df_test.apply(q1_freq_test, axis=1, raw=True)
df_test['q2_freq'] = df_test.apply(q2_freq_test, axis=1, raw=True)
print('Basic features created')

# Load features from previous work on feature_tm and feature_nlp
import pandas as pd

df_feature = pd.read_csv('Features/feature_tm+nlp.csv')
df_feature = df_feature.fillna(0)
df_feature_test = pd.read_csv('Features/feature_tm+nlp_test.csv')
print('Features(nlp&tm) Loaded')

df_train['id'] = df_feature['id']
df_test['test_id'] = df_feature_test['test_id']

df_train = df_train.merge(df_feature,on='id',how='left')
leak = df_train.drop(['id','qid1','qid2','question1','question2','is_duplicate'], axis=1)
df_test = df_test.merge(df_feature_test,on='test_id',how='left')
test_leak = df_test.drop(['test_id','question1','question2'], axis=1)
print('Final features created')


Basic features created
Features(nlp&tm) Loaded
Final features created


In [ ]:
# Make scaling for leaky feature (normalization)
ss = StandardScaler()
ss.fit(np.vstack((leak, test_leak)))
leaks = ss.transform(leak)  # the leaky feature, array
test_leaks = ss.transform(test_leak)  # the leaky feature_test, array
print('Normalization finished')


In [7]:
leaks

array([[-0.11478188, -0.05687322, -0.04436891, ..., -1.67102839,
        -1.4782846 , -1.4782846 ],
       [-0.11478188,  0.03837623, -0.03081647, ..., -0.20451842,
        -0.18610259, -0.18610259],
       [-0.11478188, -0.04326616, -0.05792136, ..., -0.7200645 ,
        -0.5830698 , -0.5830698 ],
       ...,
       [-0.11478188, -0.05687322, -0.05792136, ..., -0.83913556,
         0.44826012,  0.44826012],
       [-0.11478188, -0.05687322, -0.05792136, ...,  1.54412763,
         0.72736336,  0.72736336],
       [-0.11478188, -0.05687322, -0.05792136, ..., -3.97671095,
        -2.33599635, -2.33599635]])

In [8]:
# Train & Validation split
perm = np.random.permutation(len(train_data_1))
idx_train = perm[:int(len(train_data_1)*(1-VALIDATION_SPLIT_RATE))]
idx_val = perm[int(len(train_data_1)*(1-VALIDATION_SPLIT_RATE)):]

data_1_train = np.vstack((train_data_1[idx_train], train_data_2[idx_train]))
data_2_train = np.vstack((train_data_2[idx_train], train_data_1[idx_train]))
leaks_train = np.vstack((leaks[idx_train], leaks[idx_train]))
labels_train = np.concatenate((train_labels[idx_train], train_labels[idx_train]))

data_1_val = np.vstack((train_data_1[idx_val], train_data_2[idx_val]))
data_2_val = np.vstack((train_data_2[idx_val], train_data_1[idx_val]))
leaks_val = np.vstack((leaks[idx_val], leaks[idx_val]))
labels_val = np.concatenate((train_labels[idx_val], train_labels[idx_val]))

weight_val = np.ones(len(labels_val))
if re_weight:
    weight_val *= 0.471544715
    weight_val[labels_val==0] = 1.309033281


## Here i change the N_DENSE in dense layer from N_DENSE to N_DENSE/2, obviously the results are worse.
- give up if improve too little (<0.00x) or worse
- Scores: 0.17540, so **give it up**

In [10]:
# Reset the hyperparameter
VERSION = 'Temp/lstm_featured_2'

# Create embedding matrix for embedding layer, which is used in the keras.embedding weight as the initializer.
print('Preparing embedding matrix')

num_words = min(MAX_NUM_WORDS, len(word_index))+1

embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))  # the weight of Embedding layer
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
print('Null word embeddings: '.format(np.sum(np.sum(embedding_matrix, axis=1) == 0)))


# NN Model design
# Structure: (q1-embedding-lstm + q2-embedding-lstm + leaky-dense)-dense-sigmoid-result

# The embedding layer containing the word vectors
emb_layer = Embedding(
    input_dim=num_words,
    output_dim=EMBEDDING_DIM,
    weights=[embedding_matrix],
    input_length=MAX_SEQUENCE_LENGTH,
    trainable=False
)    

# LSTM layer
lstm_layer = LSTM(N_HIDDEN, dropout=DROPOUT_RATE_LSTM, recurrent_dropout=DROPOUT_RATE_LSTM)
# try bidirectional?
# LSTM_ = Bidirectional(LSTM(1024, kernel_regularizer=keras.regularizers.l2(0.2),return_sequences=False))

# Define inputs
seq1 = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
seq2 = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')

# Run inputs through embedding
emb1 = emb_layer(seq1)
emb2 = emb_layer(seq2)

# Run through LSTM layers
lstm_a = lstm_layer(emb1)
# glob1a = GlobalAveragePooling1D()(lstm_a)
lstm_b = lstm_layer(emb2)
# glob1b = GlobalAveragePooling1D()(lstm_b)

magic_input = Input(shape=(leaks.shape[1],))
# magic_dense = BatchNormalization()(magic_input)
magic_dense = Dense(int(N_DENSE/2), activation=ACTIVE_FUNC)(magic_input)  # change N_DENSE/2 to N_DENSE because of the addition of leaky features

merged = concatenate([lstm_a, lstm_b, magic_dense])
merged = BatchNormalization()(merged)
merged = Dropout(DROUPOUT_RATE_DENSE)(merged)

merged = Dense(N_DENSE, activation=ACTIVE_FUNC)(merged)
merged = BatchNormalization()(merged)
merged = Dropout(DROUPOUT_RATE_DENSE)(merged)

preds = Dense(1, activation='sigmoid')(merged)


# Add class weight, magic feature for the unbalancement of training labels.
if re_weight:
    class_weight = {0: 1.309033281, 1: 0.471544715}
else:
    class_weight = None
    

# Train the model

print('Starting the model training')

model = Model(inputs=[seq1, seq2, magic_input], outputs=preds)
model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc'])

# Summerization of model
model.summary()

# Set early stopping (large patience should be useful)
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
bst_model_path = VERSION + '.h5' 
model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)


hist = model.fit([data_1_train, data_2_train, leaks_train], labels_train, \
        validation_data=([data_1_val, data_2_val, leaks_val], labels_val, weight_val), \
        epochs=4, batch_size=2048, shuffle=True, \
        class_weight=class_weight, callbacks=[early_stopping, model_checkpoint])

model.load_weights(bst_model_path) # sotre model parameters in .h5 file
bst_val_score = min(hist.history['val_loss'])



Preparing embedding matrix
Null word embeddings: 
Starting the model training
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 60)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 60)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 60, 300)      36150000    input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
input_3 (Input

In [11]:
# Make the submission

print('Making the submission')

preds = model.predict([test_data_1, test_data_2, test_leaks], batch_size=8192, verbose=1)
preds += model.predict([test_data_2, test_data_1, test_leaks], batch_size=8192, verbose=1)
preds /= 2

submission = pd.DataFrame({'test_id':test_ids, 'is_duplicate':preds.ravel()})
submission.to_csv('Models/lstm_featured_2.csv', index=False)

Making the submission
2345796/2345796 [==============================] - 2679s 1ms/step


## Here i add one more dense layer
- just look at the first epoch, give up if improve too little (<0.00x) or worse
- in magic dense, scores: 0.17125, not better, **give it up**.
- in merge dense, worse, **give it up**.


In [ ]:
# Reset one hyperparameter

In [11]:
# Reset the hyperparameter
VERSION = 'Temp/lstm_featured_3'

# Create embedding matrix for embedding layer, which is used in the keras.embedding weight as the initializer.
print('Preparing embedding matrix')

num_words = min(MAX_NUM_WORDS, len(word_index))+1

embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))  # the weight of Embedding layer
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
print('Null word embeddings: '.format(np.sum(np.sum(embedding_matrix, axis=1) == 0)))
print('Start modeling')


# NN Model design
# Structure: (q1-embedding-lstm + q2-embedding-lstm + leaky-dense)-dense-sigmoid-result

# The embedding layer containing the word vectors
emb_layer = Embedding(
    input_dim=num_words,
    output_dim=EMBEDDING_DIM,
    weights=[embedding_matrix],
    input_length=MAX_SEQUENCE_LENGTH,
    trainable=False
)    

# LSTM layer
lstm_layer = LSTM(N_HIDDEN, dropout=DROPOUT_RATE_LSTM, recurrent_dropout=DROPOUT_RATE_LSTM)
# try bidirectional?
# LSTM_ = Bidirectional(LSTM(1024, kernel_regularizer=keras.regularizers.l2(0.2),return_sequences=False))

# Define inputs
seq1 = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
seq2 = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')

# Run inputs through embedding
emb1 = emb_layer(seq1)
emb2 = emb_layer(seq2)

# Run through LSTM layers
lstm_a = lstm_layer(emb1)
# glob1a = GlobalAveragePooling1D()(lstm_a)
lstm_b = lstm_layer(emb2)
# glob1b = GlobalAveragePooling1D()(lstm_b)

magic_input = Input(shape=(leaks.shape[1],))
# magic_dense = BatchNormalization()(magic_input)
magic_dense = Dense(int(N_DENSE), activation=ACTIVE_FUNC)(magic_input)  
magic_dense = Dense(int(N_DENSE/2), activation=ACTIVE_FUNC)(magic_dense)  # change

merged = concatenate([lstm_a, lstm_b, magic_dense])
merged = BatchNormalization()(merged)
merged = Dropout(DROUPOUT_RATE_DENSE)(merged)

merged = Dense(N_DENSE, activation=ACTIVE_FUNC)(merged)
merged = BatchNormalization()(merged)
merged = Dropout(DROUPOUT_RATE_DENSE)(merged)

preds = Dense(1, activation='sigmoid')(merged)


# Add class weight, magic feature for the unbalancement of training labels.
if re_weight:
    class_weight = {0: 1.309033281, 1: 0.471544715}
else:
    class_weight = None
    

# Train the model

print('Starting the model training')

model = Model(inputs=[seq1, seq2, magic_input], outputs=preds)
model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc'])

# Summerization of model
model.summary()

# Set early stopping (large patience should be useful)
early_stopping =EarlyStopping(monitor='val_loss', patience=10)
bst_model_path = VERSION + '.h5' 
model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)


hist = model.fit([data_1_train, data_2_train, leaks_train], labels_train, \
        validation_data=([data_1_val, data_2_val, leaks_val], labels_val, weight_val), \
        epochs=4, batch_size=2048, shuffle=True, \
        class_weight=class_weight, callbacks=[early_stopping, model_checkpoint])

model.load_weights(bst_model_path) # sotre model parameters in .h5 file
bst_val_score = min(hist.history['val_loss'])



Preparing embedding matrix
Null word embeddings: 
Start modeling
Starting the model training
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 60)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 60)           0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 44)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 60, 300)      36150000    input_1[0][0]                    

In [12]:
# Make the submission

print('Making the submission')

preds = model.predict([test_data_1, test_data_2, test_leaks], batch_size=8192, verbose=1)
preds += model.predict([test_data_2, test_data_1, test_leaks], batch_size=8192, verbose=1)
preds /= 2

submission = pd.DataFrame({'test_id':test_ids, 'is_duplicate':preds.ravel()})
submission.to_csv('Models/lstm_featured_3.csv', index=False)

Making the submission
2345796/2345796 [==============================] - 4978s 2ms/step


In [10]:
# Reset the hyperparameter
VERSION = 'Temp/lstm_featured_3_1'
# Create embedding matrix for embedding layer, which is used in the keras.embedding weight as the initializer.
print('Preparing embedding matrix')

num_words = min(MAX_NUM_WORDS, len(word_index))+1

embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))  # the weight of Embedding layer
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
print('Null word embeddings: '.format(np.sum(np.sum(embedding_matrix, axis=1) == 0)))
print('Start modeling')
print('Start modeling')
# NN Model design
# Structure: (q1-embedding-lstm + q2-embedding-lstm + leaky-dense)-dense-sigmoid-result

# The embedding layer containing the word vectors
emb_layer = Embedding(
    input_dim=num_words,
    output_dim=EMBEDDING_DIM,
    weights=[embedding_matrix],
    input_length=MAX_SEQUENCE_LENGTH,
    trainable=False
)    

# LSTM layer
lstm_layer = LSTM(N_HIDDEN, dropout=DROPOUT_RATE_LSTM, recurrent_dropout=DROPOUT_RATE_LSTM)
# try bidirectional?
# LSTM_ = Bidirectional(LSTM(1024, kernel_regularizer=keras.regularizers.l2(0.2),return_sequences=False))

# Define inputs
seq1 = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
seq2 = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')

# Run inputs through embedding
emb1 = emb_layer(seq1)
emb2 = emb_layer(seq2)

# Run through LSTM layers
lstm_a = lstm_layer(emb1)
# glob1a = GlobalAveragePooling1D()(lstm_a)
lstm_b = lstm_layer(emb2)
# glob1b = GlobalAveragePooling1D()(lstm_b)

magic_input = Input(shape=(leaks.shape[1],))
# magic_dense = BatchNormalization()(magic_input)
magic_dense = Dense(int(N_DENSE/2), activation=ACTIVE_FUNC)(magic_input)  


merged = concatenate([lstm_a, lstm_b, magic_dense])
merged = BatchNormalization()(merged)
merged = Dropout(DROUPOUT_RATE_DENSE)(merged)

merged = Dense(N_DENSE, activation=ACTIVE_FUNC)(merged)
merged = BatchNormalization()(merged)
merged = Dropout(DROUPOUT_RATE_DENSE)(merged)

preds = Dense(1, activation='sigmoid')(merged)


# Add class weight, magic feature for the unbalancement of training labels.
if re_weight:
    class_weight = {0: 1.309033281, 1: 0.471544715}
else:
    class_weight = None
    

# Train the model

print('Starting the model training')

model = Model(inputs=[seq1, seq2, magic_input], outputs=preds)
model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc'])

# Summerization of model
model.summary()

# Set early stopping (large patience should be useful)
early_stopping =EarlyStopping(monitor='val_loss', patience=10)
bst_model_path ='lstm_featured_4.h5' 
model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)


hist = model.fit([data_1_train, data_2_train, leaks_train], labels_train, \
        validation_data=([data_1_val, data_2_val, leaks_val], labels_val, weight_val), \
        epochs=4, batch_size=2048, shuffle=True, \
        class_weight=class_weight, callbacks=[early_stopping, model_checkpoint])

model.load_weights(bst_model_path) # sotre model parameters in .h5 file
bst_val_score = min(hist.history['val_loss'])



Preparing embedding matrix
Null word embeddings: 
Start modeling
Start modeling
Starting the model training
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 60)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 60)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 60, 300)      36150000    input_1[0][0]                    
                                                                 input_2[0][0]                    
___________________________________________________________________________________

In [11]:
# Make the submission

print('Making the submission')

preds = model.predict([test_data_1, test_data_2, test_leaks], batch_size=8192, verbose=1)
preds += model.predict([test_data_2, test_data_1, test_leaks], batch_size=8192, verbose=1)
preds /= 2

submission = pd.DataFrame({'test_id':test_ids, 'is_duplicate':preds.ravel()})
submission.to_csv('Models/lstm_featured_3_1.csv', index=False)

Making the submission
2345796/2345796 [==============================] - 2858s 1ms/step


## Here i change some hyperparameters
- just look at the first epoch
- n-hidden
- dense_num

In [2]:
import pandas as pd
df = pd.read_csv('Models/lstm_featured_1.csv')
df = df.is_duplicate[-4:]
df.values

array([6.1559826e-03, 1.3150275e-04, 5.4044873e-03, 2.3005173e-01])

In [ ]:
# Wen Ze's part, he will do these experiments based on lstm.ipynb